##2. What type of problem are we trying to solve here? Classification or regression?
Classification, to identify if the patient is exposed to heart disease or not.

In [0]:
#Question3

import numpy as py
import pandas as pd

#Mount google drive
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

#Read the file from google drive
data = pd.read_csv( 'gdrive/My Drive/Machine Learning Playground/heart.csv' )
print (data)

In [0]:
# Normalizing data
# This step is really useful on improving model performance
from sklearn.preprocessing import StandardScaler
cols_to_norm = ['age','chol','trestbps','thalach','oldpeak']
data[cols_to_norm] = StandardScaler().fit_transform(data[cols_to_norm])

# Create predictors
predictors = data.drop(['target'], axis=1).values
n_cols = predictors.shape[1] 
  
# Create target
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(data.target)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, 
                                                    test_size=0.3 )

# Build and train the model
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

# Input layer
model = keras.Sequential() 
model.add(layers.Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(layers.Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(layers.Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(layers.Dense(100, activation='relu', input_shape = (n_cols,))) 

# Output layer
model.add(layers.Dense(1, activation='sigmoid')) 

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

#Define early stopping to stop training if model didn't improved for 2 epochs
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=5, monitor = 'acc')

# Fit the model
model.fit(X_train, y_train, epochs=40, verbose=1, callbacks=[early_stopping_monitor] )

# Evaluate the model
model.evaluate(X_test, y_test )

##Epoch 20/40
Epoch 20/40
212/212 [==============================] - 0s 157us/sample - loss: 0.0571 - acc: 0.9906
Accuracy at Epoch20 is 0.9906 or 99.06%

Due to the randomness during the training of model, the accuracy might be different

In [113]:
# fix random seed for reproducibility
seed = 7
py.random.seed(seed)
cvscores = []

# define 10-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for train, test in kfold.split(predictors, target):
  # Input layer
  model = keras.Sequential() 
  model.add(layers.Dense(100, activation='relu', input_shape = (n_cols,))) 
  model.add(layers.Dense(100, activation='relu')) 

  # Output layer
  model.add(layers.Dense(1, activation='sigmoid')) 

  # Compile model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

  # Fit the model
  model.fit(predictors[train], target[train], epochs=40,  verbose=False )
  # evaluate the model
  scores = model.evaluate(predictors[test], target[test], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (py.mean(cvscores), py.std(cvscores)))

acc: 78.69%
acc: 81.97%
acc: 78.69%
acc: 86.67%
acc: 81.67%
81.54% (+/- 2.92%)


#4. What is underfitting and overfitting?
Underfitting : the model is not train enough and unable to identify / detect any useful pattern from the input data
Overfitting  : the model is over trained / fully adapted to the input data, thus it is very sensitive to the changes of input data and not really useful in prediction where the input data is differ to the training data.  

#5. Explain how you can reduce the overfitting of the model
To reduce overfitting of data, K-fold cross validation / splitting the data into training and validation will help to validate the model. By doing K-fold cross validation, data will be splitted into multiple set of test & train and repeated multiple times. Each time, the test & train data will be shuffled, thus preventing the model from overfitting by introducing data in various form.

Splitting the training data into training and validation only application at big dataset,